# 4 Potential Outcomes Model

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math
import random

In [2]:
np.random.seed(91)
random.seed(91)

## 4.0.1 Statistical Inference

In [3]:
url = 'https://github.com/scunning1975/mixtape/raw/master/yule.dta'
df = pd.read_stata(url)

In [4]:
df.head()

,location,paup,outrelief,old,pop
0,Kensington,27,5,104,136
1,Paddington,47,12,115,111
2,Fulham,31,21,85,174
3,Chelsea,64,21,81,124
4,St. George’s,46,18,113,96


In [5]:
# regression
X = df[['outrelief', 'old', 'pop']]
X = sm.add_constant(X)
results = sm.OLS(df.paup, X).fit()

In [6]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   paup   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     21.49
Date:                Sun, 14 Mar 2021   Prob (F-statistic):           2.00e-07
Time:                        21:05:35   Log-Likelihood:                -115.47
No. Observations:                  32   AIC:                             238.9
Df Residuals:                      28   BIC:                             244.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         63.1877     27.144      2.328      0.0

## 4.1.4 Independence Assumption

In [7]:
# compare to independence
sdo_list = list()

for sim in range(10000):
    y1 = 7, 5, 5, 7, 4, 10, 1, 5, 3, 9
    y1 = np.array(y1)
    y0 = 1, 6, 1, 8, 2, 1, 10, 6, 7, 8
    y0 = np.array(y0)

    d = np.concatenate([np.ones(5), np.zeros(5)])
    random.shuffle(d)
    y = d*y1 + (1-d)*y0

    # means
    sy1 = np.mean(y[d==1])
    sy0 = np.mean(y[d==0])
    sdo = sy1 - sy0
    
    sdo_list.append(sdo)
    
    

In [8]:
print('SDO', np.mean(sdo_list))

SDO 0.59042


In [9]:
ate = np.mean(y1 - y0)
print('ATE', ate)

ATE 0.6


## 4.2.1 Lady Tasting Tea

In [10]:
from itertools import combinations

In [11]:
# compare to tea

# 8 cups of tea, assume she guesses the first four cups
guess = (1,2,3,4)

n_combos = 0
correct = 0
for truth in combinations(range(1,9), 4): # possible combos of cups with tea then milk 
    n_combos += 1 # add one to count of combinations
    if truth == guess:
        correct += 1

In [12]:
print('probability of guessing the truth', correct/n_combos)

probability of guessing the truth 0.014285714285714285


## 4.2.4 Example

In [13]:
# compare to ri

url = 'https://github.com/scunning1975/mixtape/raw/master/ri.dta'
df = pd.read_stata(url, index_col = 'name')

In [14]:
test_stat_helper = df.groupby('d').y.mean()
observed_test_stat = test_stat_helper[1] - test_stat_helper[0]
print(observed_test_stat)

1.0


In [15]:
# sharp null
#sharp_null = df.replace('.',np.nan) 
#sharp_null = sharp_null.bfill(axis = 1).ffill(axis = 1) # set y0 = y1 by filling nulls

# randomly assign d = 1
assignment_table = pd.DataFrame()
for key, assignment in enumerate(combinations(df.index, 4)):
    control = [x for x in df.index if x not in assignment] # those not in treatment group
    mean_treated = df.loc[assignment,'y'].mean()
    mean_u = df.loc[control, 'y'].mean()
    
    test_stat = mean_treated - mean_u
    
    # dicts
    dict1 = dict(zip(assignment,np.ones(4)))
    dict2 = dict(zip(control, np.zeros(4)))
    dict3 = {'T': test_stat}
    dict1.update(dict2)
    dict1.update(dict3)
    row = pd.DataFrame(data = dict1, index = [key])
    
    assignment_table = assignment_table.append(row)

In [16]:
# do it again with permutations instead of combinations

from itertools import permutations

y_vec = df.y.values 

d = np.concatenate( [np.ones(4), (-1)*np.ones(4)] )
t_stats = list()
for d_vec in permutations(d):    
    
    t = np.dot(y_vec, d_vec) / 4 # signed value
    t_stats.append(t)